# Linear Regression

### Q1

In [149]:
import numpy as np
import math
import operator
from scipy.linalg import svd
import scipy.stats as ss
import matplotlib.pyplot as plt
import scipy.linalg as la

x = np.array([[-2], [-5], [-3], [0], [-8], [-2], [1], [5], [-1], [6]])
y = np.array([[1], [-4], [1], [3], [11], [5], [0], [-1], [-3], [1]])

def lse_1(x,y):
    biasF = np.ones(((len(x)), 1))
    X = np.hstack((biasF, x))

    dotX = np.linalg.inv(np.dot(X.T,X))
    dotY = np.dot(X.T,y)

    w = np.dot(dotX, dotY)

    return w
    

w = lse_1(x,y)

Y = np.zeros(shape=(10,1))

for j in range(len(y)):
    Y[j] = (w[1]*x[j] + w[0])

print("Y_pred: \n", Y)

def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2))

rootmean = rmse(Y, y)
print("RMSE: ", rmse(Y, y))


Y_pred: 
 [[ 1.85394655]
 [ 3.0919826 ]
 [ 2.26662523]
 [ 1.02858919]
 [ 4.33001865]
 [ 1.85394655]
 [ 0.6159105 ]
 [-1.03480423]
 [ 1.44126787]
 [-1.44748291]]
RMSE:  3.7013259176662716


In [150]:
### Q2

In [151]:
# to do

### Q3

In [183]:
import cv2
from cv2 import VideoWriter, VideoWriter_fourcc
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from collections import Counter
import glob
from matplotlib.image import imread
from enum import Enum


class Sex(Enum):
    male, female = range(2)

class Smoker(Enum):
    no, yes = range(2)
    
class Region(Enum):
    southwest, southeast, northwest, northeast = range(4)


np.random.seed(0)
imp_data = np.genfromtxt('insurance.csv', delimiter=',', encoding='utf8', dtype=np.str)
# dataN = data.astype(np.float)

# Separates header and data
feature_name, data = np.vsplit(imp_data, [1])

x, y = np.hsplit(data, [-1])

n = len(x)

# Rows shuffled
np.random.shuffle(x)

# Calculates array index for splitting
spltIdx = int((2/3)*n)

# Training-validation data split
x_train, x_test, y_train, y_test = x[:spltIdx,:], x[spltIdx:,:], y[:spltIdx,:], y[spltIdx:,:]

print(x_train)

def preprocessCat(dataMain):
    '''Changes categorical features to enumerated ones'''

    data = np.copy(dataMain)

    for i in range(len(data)):
        for s in Sex:
            if s.name == data[i][1]:
                data[i][1] = s.value
                break
        
        for sm in Smoker:
            if sm.name == data[i][4]:
                data[i][4] = sm.value
                break

        for r in Region:
            if r.name == data[i][5]:
                data[i][5] = r.value
                break

    return data

def lse(x,y, addBias):
    '''Calculates weights using LSE'''

    dataX = x.astype(np.float)
    dataY = y.astype(np.float)

    if addBias:
        biasF = np.ones(((len(dataX)), 1))
        X = np.hstack((biasF, dataX))
    else:
        X = np.array(dataX)


    dotX = np.linalg.inv(np.dot(X.T,X))
    dotY = np.dot(X.T,dataY)

    w = np.dot(dotX, dotY)

    return w



def rmse(w, x, y):
    '''Calculates RMSE'''

    Y = np.zeros(shape=((len(y_train)),1))
    for j in range(len(y_train)):
        Y[j] =  w[1] * x[0] + w[2] * x[1] + w[3] * x[2] + w[4] * x[3] + w[5] * x[4] + w[0]
    return np.sqrt(np.mean((predictions-targets)**2))

# Change categorical features to enumerated ones (NO BIAS)
x_t_preC = preprocessCat(x_train)
w_t_preC = lse(x_t_preC, y_train, False)



# for j in range(len(y_train)):
#     Y[j] = w_t_preC[1]*x[j] + w[0])

print(w_t_preC)




[['52' 'male' '30.2' '1' 'no' 'southwest']
 ['47' 'female' '29.37' '1' 'no' 'southeast']
 ['48' 'male' '40.565' '2' 'yes' 'northwest']
 ...
 ['49' 'male' '30.9' '0' 'yes' 'southwest']
 ['19' 'male' '34.8' '0' 'yes' 'southwest']
 ['58' 'male' '36.08' '0' 'no' 'southeast']]
[[  43.28486778]
 [  26.2419698 ]
 [ 316.19111346]
 [  -5.82274963]
 [1785.2134203 ]
 [ 752.16256354]]
